# Get Training Data

In [1]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

/home/VD/fruitao/drone/logic_guided_prompt/code /home/VD/fruitao/drone/logic_guided_prompt


In [2]:
from logic.description_logic import *
from utils.load_sensor_data import *
from data_augmentation.text_augmentation import *

/home/VD/fruitao/drone/logic_guided_prompt/code/data_augmentation /home/VD/fruitao/drone/logic_guided_prompt/code


2023-06-06 00:43:31.824921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 00:43:32.045969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-06 00:43:32.045991: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-06 00:43:32.819193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinf

In [3]:
print(WARNING_SITUATION_EXAMPLES)

{'flying_not_normal_frame': ["[{'Normal_frame': false}]", 'The drone is flying with a damaged frame.'], 'near_moving_object': ["[{'time_stamp': '0:01', 'name': 'car', 'Distance': 3.0}]", "(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m."], 'reachable_inpath': ["[{'time_stamp': '0:05', 'name': 'car', 'Distance': 0.3}]", "(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."], 'flying_empty_battery': ["[{'Bettery_level': 20}]", 'The flying drone is runing out of battery with only 20% charge.'], 'low_battery_strong_wind': ["[{'Bettery_level': 30, 'WindSpeed (m/s)': 18}]", 'The drone is flying in a strong wind of 18m/s with a low battery level at 30%.'], 'near_object_inexperienced': ["[{'time_stamp': '0:00', 'name': 'Bird1', 'Distance': 0.5}, {'experienced': false}]", "(0:00) The drone is facing the risk of physical damage. It's flying too close to Bird1 at a dista

In [4]:
# old_format_example=pd.ExcelFile(r"../text_annotation/rural.xlsx")

dataset_file_li=[r"../../text_annotation/desert.xlsx", r"../../text_annotation/disturbance.xlsx", r"../../text_annotation/factory.xlsx", r"../../text_annotation/island.xlsx", 
                 r"../../text_annotation/misc.xlsx", r"../../text_annotation/ocean.xlsx", r"../../text_annotation/rural.xlsx", r"../../text_annotation/urban.xlsx"
]

# dataset_file_li=[ r"../text_annotation/rural.xlsx"]

old_data_dic=dict()

for dataset_file in dataset_file_li:
    print("----processing file: {}----".format(dataset_file))
    
    old_format_example=pd.ExcelFile(dataset_file)

    ts_converters={'name': str, 'Type': str, 'Moving': str2bool, 'InPath': str2bool, 'time_stamp': str}
    status_converters={'PilotExperienced': str2bool, 'Low_visibility': str2bool, 'Normal_frame': str2bool,
                      'weather': str, 'upside_down': str2bool, 'good_motor_condition': str2bool,
                      'going_backwards': str2bool, 'indoor': str2bool, 'waterproof drone?': str2bool,
                      'flying over': str, 'criticality': str, 
                      'RiskOfPhysicalDamage': str, 'RiskOfInternalDamage': str2bool,
                      'RiskOfHumanDamage': str2bool, 'LostConnection': str2bool}
    text_converters={'Link':str, 'Text1':str, 'Text2':str}

    old_format_ts=pd.read_excel(old_format_example, sheet_name='timestep', converters=ts_converters)
    old_format_status=pd.read_excel(old_format_example, sheet_name='status', converters=status_converters)
    old_format_text=pd.read_excel(old_format_example, sheet_name='text', converters=text_converters)

    print("---loading sensor data---")
    load_ts_to_dict(old_format_ts, old_data_dic)
    load_status_to_dict(old_format_status, old_data_dic)
    load_text_to_dict(old_format_text, old_data_dic)

    # print("---generating templates---")
    for link, data in old_data_dic.items():
        templates, related_status_dic, related_timestep_dic = gen_templates(data)
        old_data_dic[link]['templates'] = templates
        old_data_dic[link]['related_status_dic'] = related_status_dic
        old_data_dic[link]['related_timestep_dic'] = related_timestep_dic


----processing file: ../../text_annotation/desert.xlsx----
---loading sensor data---
--gen_templates-- related_status_dic:{'flying_not_normal_frame': [{'Normal_frame': False}]}
--gen_templates-- template:The drone is flying with a damaged frame.
templates:{'flying_not_normal_frame': ['The drone is flying with a damaged frame.'], 'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:05) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.3m."], 'reachable_inpath': ["(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."]}
templates:{'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:03) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.2m."], 'reachable_inpath': 

In [5]:
# print("old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['timestep'][0]['time_stamp']:{}".format(old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['timestep'][0]['time_stamp']))
print("len(old_data_dic):{}".format(len(old_data_dic)))
print("--['text'][0]:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['text'][0]))
print("--['related_status_dic']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['related_status_dic']))
print("--['related_timestep_dic']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['related_timestep_dic']))
print("--['templates']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['templates']))

len(old_data_dic):300
--['text'][0]:{'Text1': "Risk of Physical Damage! The drone has a damaged frame and empty battery. It's flying at an altitude of 5m. [SEP]\n(0:01) Risk of Physical Damage! There are two gravestones in the drone's flight path at a distance of 3m.  [SEP] \n(0:05) Risk of Physical Damage! Now there's a castle in the drone's flight path at a distance of 1m. [SEP]", 'Text2': "Risk of physical damage! The drone is flying with an empty battery. The frame is also damaged and the altitude is 5m. [SEP]\n(0:01) Risk of physical damage! There are two gravestones 3m away and in the drone's path. [SEP]\n(0:05) Risk of physical damage! The drone is flying toward a castle in its path which is only 1m away. [SEP]"}
--['related_status_dic']:{'flying_empty_battery': [{'Battery_level': 0}], 'flying_not_normal_frame': [{'Normal_frame': False}]}
--['related_timestep_dic']:{'reachable_inpath': [{'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name

In [6]:
print(old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing'].keys())

dict_keys(['timestep', 'status', 'text', 'templates', 'related_status_dic', 'related_timestep_dic'])


In [7]:
import json

link_li=[]
text_li=[]
dl_name_li=[]
templates_li=[]
related_sensor_data_li=[]
related_status_dic_li=[]
related_timestep_dic_li=[]
dl_related_sensor_data_mapping_li=[]

oneshot_examples_li=[]
text_related_sensor_data_mapping_li=[]

# templates_nested_li=[]
# related_status_dic_nested_li=[]
# related_timestep_dic_nested_li=[]
# related_sensor_data_nested_li=[]

# related_sensor_data_li as input and templates_li as label
dl_inds={
    "out_of_range_low_battery": 0,
    "flying_empty_battery": 0,
    "empty_battery_water_proof_flying_water": 0,
    "flying_not_normal_frame": 0,
    "low_battery_low_temperature": 0,
    "low_battery_strong_wind": 0,
    "low_battery_high_altitude": 0,
    "water_proof_precipitation": 0,
    "flying_water_low_altitude": 0,
    "rain_strong_wind": 0,
    "upsidedown_inexperienced": 0,
    "extreme_weather_inexperienced": 0,
    "gloomy_high_altitude":0,
    "low_altitude_fast_speed": 0,
    
    "indoor_near_human": 0,
    "low_visibility_near_object": 0,
    "near_fragile_object_near_human": 0,
    "near_object_inexperienced": 0,
    "near_moving_object": 0,
    "reachable_inpath": 0,
    "very_close_human": 0,
    "very_close_lamp": 0,
    "very_close_object": 0,

}

for link, value in old_data_dic.items():
#     print("value:{}".format(value))
    link_li.append(link)
    if 'text' not in value:
        text_li.append([])
    else:    
        text_li.append(value['text']) 
    
    text_related_sensor_data_mapping=list()
    text_related_sensor_data_mapping_li.append(text_related_sensor_data_mapping)
            
    for dl_name, template_value in value['templates'].items():
        templates_li.extend(template_value)
        
        if dl_name in value['related_status_dic']:
            related_status_dic_li.extend(value['related_status_dic'][dl_name])
            related_sensor_data_li.extend(value['related_status_dic'][dl_name])
            
            len_new_status=len(value['related_status_dic'][dl_name])
            len_current_sensor=len(related_sensor_data_li)
            text_related_sensor_data_mapping.append(list(range(len_current_sensor-len_new_status, len_current_sensor)))
            
            for i in range(len(value['related_status_dic'][dl_name])):
                oneshot_examples_li.append(WARNING_SITUATION_EXAMPLES[dl_name])
                
            
        elif dl_name in value['related_timestep_dic']:
            related_timestep_dic_li.extend(value['related_timestep_dic'][dl_name])
            related_sensor_data_li.extend(value['related_timestep_dic'][dl_name])
            
            len_new_timestep=len(value['related_timestep_dic'][dl_name])
            len_current_sensor=len(related_sensor_data_li)
            text_related_sensor_data_mapping.append(list(range(len_current_sensor-len_new_timestep, len_current_sensor)))
            
            for i in range(len(value['related_timestep_dic'][dl_name])):
                oneshot_examples_li.append(WARNING_SITUATION_EXAMPLES[dl_name])
                
            
        else:
            print("WHAT?")
            
    
#         if not dl_inds[dl_name] and (dl_name in value['related_status_dic']):
#             print('"{}": ["{}", "{}"]'.format(dl_name, json.dumps(value['related_status_dic'][dl_name]), template_value[0]))
#             dl_inds[dl_name]=1
            
#         if not dl_inds[dl_name] and (dl_name in value['related_timestep_dic']):
#             print('"{}": ["{}", "{}"]'.format(dl_name, json.dumps(value['related_timestep_dic'][dl_name]), template_value[0]))
#             dl_inds[dl_name]=1
        
related_sensor_data_li=[json.dumps(dic) for dic in related_sensor_data_li]

print("related_sensor_data_li", len(related_sensor_data_li), related_sensor_data_li[0])
print("oneshot_examples_li", len(oneshot_examples_li), oneshot_examples_li[0])
print("templates_li", len(templates_li), templates_li[0])
print("text_li", len(text_li), text_li[0])
print("text_related_sensor_data_mapping_li", len(text_related_sensor_data_mapping_li), text_related_sensor_data_mapping_li[0])

# print("text_related_sensor_data_mapping_li", len(text_related_sensor_data_mapping_li), text_related_sensor_data_mapping_li)
mapping_example_id=2
mapping_example=text_related_sensor_data_mapping_li[mapping_example_id]
sensor_mapping=[related_sensor_data_li[sensor_id] for _, sensor_ids in enumerate(mapping_example) for sensor_id in sensor_ids]
oneshot_mapping=[oneshot_examples_li[sensor_id] for _, sensor_ids in enumerate(mapping_example) for sensor_id in sensor_ids]
print(len(mapping_example), mapping_example)
print(len(sensor_mapping), sensor_mapping)
print(len(oneshot_mapping), oneshot_mapping)
print(len(text_li[mapping_example_id]), text_li[mapping_example_id])

related_sensor_data_li 1652 {"Normal_frame": false}
oneshot_examples_li 1652 ["[{'Normal_frame': false}]", 'The drone is flying with a damaged frame.']
templates_li 1652 The drone is flying with a damaged frame.
text_li 300 [{'Text1': "Risk of physical damage! The drone has a damaged frame. [SEP]\n(0:05) Risk of physical damage! There's a car in the drone's flight path at a distance of .3m. [SEP]", 'Text2': "Risk of physical damage! The drone's frame is in need of repair. [SEP]\n(0:05) Risk of physical damage! In .3m there's a car in the flight path. [SEP]"}]
text_related_sensor_data_mapping_li 300 [[0], [1, 2], [3]]
3 [[7], [8, 9], [10]]
4 ['{"Normal_frame": false}', '{"time_stamp": "0:03", "name": "car", "Distance": 1.0}', '{"time_stamp": "0:06", "name": "car", "Distance": 0.2}', '{"time_stamp": "0:06", "name": "car", "Distance": 0.2}']
4 [["[{'Normal_frame': false}]", 'The drone is flying with a damaged frame.'], ["[{'time_stamp': '0:01', 'name': 'car', 'Distance': 3.0}]", "(0:01) T

In [8]:
import math
import random

def train_test_split(all_li, train_ratio=0.6):
    all_len=len(all_li)
    train_len=math.ceil(train_ratio*all_len)
    
    return all_li[:train_len], all_li[train_len:]

train_text, test_text=train_test_split(text_li)
train_mapping, test_mapping=train_test_split(text_related_sensor_data_mapping_li)

train_sensor_data=[]
train_oneshot_data=[]
test_sensor_data=[]
test_oneshot_data=[]

for text_id, text in enumerate(train_text):
#     print("text_id:{}, text:{}".format(text_id, text))
#     for mapping_id, sensor_ids in enumerate(text_related_sensor_data_mapping_li[text_id]):
#         print("mapping_id:{}, sensor_ids:{}".format(mapping_id, sensor_ids))
#         for sensor_id in sensor_ids:
#             print(related_sensor_data_li[sensor_id])
    
    train_sensor_data.append([related_sensor_data_li[sensor_id] for _, sensor_ids in enumerate(train_mapping[text_id]) for sensor_id in sensor_ids])
    train_oneshot_data.append([oneshot_examples_li[sensor_id] for _, sensor_ids in enumerate(train_mapping[text_id]) for sensor_id in sensor_ids])

for text_id, text in enumerate(test_text):
#     text_id+=len(train_text)
    test_sensor_data.append([related_sensor_data_li[sensor_id] for _, sensor_ids in enumerate(test_mapping[text_id]) for sensor_id in sensor_ids])
    test_oneshot_data.append([oneshot_examples_li[sensor_id] for _, sensor_ids in enumerate(test_mapping[text_id]) for sensor_id in sensor_ids])
    
print(len(related_sensor_data_li), len(train_sensor_data), len(test_sensor_data), test_sensor_data[5])
print(len(text_li), len(train_text), len(test_text), test_text[5])
print(len(oneshot_examples_li), len(train_oneshot_data), len(test_oneshot_data))
print(len(test_oneshot_data[5]))
for data in test_oneshot_data[5]:
    print(data)

1652 180 120 ['{"time_stamp": "0:01", "name": "industrial water tank 1", "Distance": 2.0}', '{"time_stamp": "0:03", "name": "industrial water tank 1", "Distance": 0.5}', '{"time_stamp": "0:01", "name": "industrial water tank 1", "Distance": 2.0}']
300 180 120 [{'Text1': "(0:01) Risk of physical damage! There's an industrial water tank in the drone's flight path at a distance of 2m.  [SEP]\n(0:03) Risk of physical damage! Now the industrial water tank is only .5m away. [SEP]", 'Text2': "(0:01) Risk of physical damage! There's an industrial water tank in the drone's path only 2m away. [SEP]\n(0:03) Risk of physical damage! Now the tank isn't in the path but is only .5m away. [SEP]"}]
1652 180 120
3
["[{'time_stamp': '0:00', 'name': 'Bird1', 'Distance': 0.5}, {'experienced': false}]", "(0:00) The drone is facing the risk of physical damage. It's flying too close to Bird1 at a distance of 0.5m, and the pilot is not experienced."]
["[{'time_stamp': '0:00', 'name': 'Bird1', 'Distance': 0.5},

In [9]:
# mapping{'flying_empty_battery':[0], 'flying_not_normal_frame':[1], 'reachable_inpath':[2, 3, 4]}
# input: [{'Battery_level': 0}, {'Normal_frame': False}, {'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name': 'gravestone 2', 'Distance': 3.0}, {'time_stamp': '0:05', 'name': 'castle ', 'Distance': 1.0}]
# output: 'The flying drone is runing out of battery with only 0% charge.'

print(len(related_sensor_data_li), related_sensor_data_li[0], type(related_sensor_data_li[0]))
print(len(templates_li), templates_li[0], type(templates_li[0]))
# print(dl_related_sensor_data_mapping_li[2])

1652 {"Normal_frame": false} <class 'str'>
1652 The drone is flying with a damaged frame. <class 'str'>


In [10]:
from torch.utils.data import Dataset

class drone_dataset(Dataset):
    def __init__(self, data_dic):
        self.data_dic=data_dic
        
    
    def __len__(self):
        return len(self.data_dic)
    
    def __getitem__(self, idx):
        return data_dic['related_status_dic'][0], data_dic['related_timestep_dic'], data_dic['text'][0]
    
    

# Augmented Prompt

In [11]:
!nvidia-smi

Tue Jun  6 00:43:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   45C    P0    55W / 250W |  31520MiB / 32768MiB |     76%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   

In [12]:
import torch

device=torch.device("cuda:6")

In [13]:
import math

from tqdm import tqdm
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.nn as nn
from sklearn.metrics import accuracy_score
from nltk.translate import meteor_score
import torch.nn.functional as F

In [14]:
n_tokens = 20
initialize_from_vocab = True

In [15]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
if torch.cuda.is_available():
    model = model.to(device)

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:173: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [22]:
# Todo: Use dataset instead.
def prefix_one_shot(input_str, one_shot_input, one_shot_output):
    return "input: {one_shot_input}, output: {one_shot_output}\n input:{input_str}, output:".format(
        one_shot_input=one_shot_input, one_shot_output=one_shot_output, input_str=input_str
    )


def generate_data(n_tokens, x_data, y_data, oneshot_data, y_x_mapping, tokenizer):

    def yield_data(x_batch, y1_batch, y2_batch):
            
        x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
        y1 = torch.nn.utils.rnn.pad_sequence(y1_batch, batch_first=True)
        y2 = torch.nn.utils.rnn.pad_sequence(y2_batch, batch_first=True)
            
        # Todo: Add prompt mask for different description logic
        m = (x > 0).to(torch.float32)
        
        decoder_input_ids = torch.full((x.shape[0], y1.shape[1]), 1)
        
        if torch.cuda.is_available():
            x = x.to(device)
            y1 = y1.to(device)
            y2 = y2.to(device)
            m = m.to(device)
            decoder_input_ids = decoder_input_ids.to(device)
        return x, y1, y2, m, decoder_input_ids

    x_batch, y1_batch, y2_batch = [], [], []

#     for map_id, mapping in enumerate(y_x_mapping):
        
#         x_batch=[x_data[sensor_id] for _, sensor_ids in enumerate(mapping) for sensor_id in sensor_ids]
#         y_batch=y_data[map_id]
#         oneshot_batch=[oneshot_data[sensor_id] for _, sensor_ids in enumerate(mapping) for sensor_id in sensor_ids]        
    
    for x_li, y_li, oneshot_li in zip(x_data, y_data, oneshot_data):
        
        for index, x in enumerate(x_li):
            
            y_1=y_li[0]['Text1']
            y_2=y_li[0]['Text2']
            
            oneshot_input, oneshot_output=oneshot_li[index]
            context=prefix_one_shot(x, oneshot_input, oneshot_output)
            
            inputs = tokenizer(context, return_tensors="pt")
#             inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        
            outputs_1=tokenizer(y_1, return_tensors="pt")
            outputs_ids_1=outputs_1['input_ids']
#             outputs_ids_1 = torch.cat([torch.full((1, n_tokens - 1), 0), outputs_1['input_ids']], 1)
            
            outputs_2=tokenizer(y_2, return_tensors="pt")
            outputs_ids_2=outputs_2['input_ids']
#             outputs_ids_2 = torch.cat([torch.full((1, n_tokens - 1), 0), outputs_2['input_ids']], 1)
            
#             print(type(inputs), inputs['input_ids'][0].size(), inputs['input_ids'].size(), 
#                   type(outputs), outputs['input_ids'][0].size(), outputs['input_ids'].size())
        
            x_batch.append(inputs['input_ids'][0][:512])
            y1_batch.append(outputs_ids_1[0][:512])
            y2_batch.append(outputs_ids_2[0][:512])
            
        yield yield_data(x_batch, y1_batch, y2_batch)

    if len(x_batch) > 0:
        yield yield_data(x_batch, y_batch, l_batch)
        x_batch, y_batch, l_batch = [], [], []

In [23]:
# model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
# tokenizer = T5Tokenizer.from_pretrained("t5-base")


ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)

# Test generation function
for x_batch, y1_batch, y2_batch, m_batch, dii_batch in generate_data(n_tokens, train_sensor_data, train_text, train_oneshot_data, train_mapping, tokenizer=tokenizer):
    
    optimizer.zero_grad()
            
    sequence_ids=model(input_ids=x_batch, attention_mask=m_batch, decoder_input_ids=dii_batch)
    
    output_ids=sequence_ids.logits[:, n_tokens:, :]
    max_pred_ids=sequence_ids.logits[:, n_tokens:, :].argmax(-1).detach()
        
    print("BEFORE max_pred_ids:{}".format((type(max_pred_ids), max_pred_ids.dtype, max_pred_ids.shape, max_pred_ids)))
    print("output_ids:{}".format((output_ids, output_ids.dtype, output_ids.shape)))

    flat_output_ids=torch.empty((0, output_ids.shape[2]), dtype=output_ids.dtype).to(device)
    flat_max_pred_ids=torch.empty((0,), dtype=max_pred_ids.dtype).to(device)
    for i, (max_pred_ids_unbatch, output_ids_unbatch) in enumerate(zip(max_pred_ids, output_ids)):
        max_pred_ids_tmp=max_pred_ids_unbatch[max_pred_ids_unbatch!=3]
        output_ids_tmp=output_ids_unbatch[max_pred_ids_unbatch!=3]
        
        flat_max_pred_ids=torch.cat((flat_max_pred_ids, max_pred_ids_tmp), 0)
        flat_output_ids=torch.cat((flat_output_ids, output_ids_tmp), 0)
    
    label_ids=torch.tensor(y1_batch[0], dtype=torch.long).to(device)
    print("decode pred_ids", tokenizer.decode(flat_max_pred_ids))
    print("decode label", tokenizer.decode(label_ids))
    
    print("AFTER flat_max_pred_ids:{}".format((type(flat_max_pred_ids), flat_max_pred_ids.dtype, flat_max_pred_ids.shape, flat_max_pred_ids)))
    print("flat_output_ids:{}".format((flat_output_ids, flat_output_ids.dtype, flat_output_ids.shape)))
    print("label_ids:{}".format((label_ids, label_ids.dtype, label_ids.shape)))
    
    max_length=max(len(flat_output_ids), len(label_ids))
    pad_width_pred=max_length-len(flat_output_ids)
    pad_width_label=max_length-len(label_ids)
    print("max_length", max_length)
    print("pad_width_pred", pad_width_pred)
    print("pad_width_label", pad_width_label)
    
    flat_output_ids=F.pad(flat_output_ids, (0, 0, 0, pad_width_pred), mode='constant', value=0)
    label_ids=F.pad(label_ids, (0, pad_width_label), mode='constant', value=0)
    
    print("flat_output_ids:{}".format((flat_output_ids, flat_output_ids.dtype, flat_output_ids.shape)))
    print("label_ids:{}".format((label_ids, label_ids.dtype, label_ids.shape)))
    
#     print("max_pred_ids", type(max_pred_ids), max_pred_ids.dtype, max_pred_ids.shape, max_pred_ids)
#     print("label", type(label), label.dtype, label.shape, label)
    
    loss=ce_loss(flat_output_ids, label_ids)
    loss.backward()
    optimizer.step()
    print("loss", loss)

    break

x_batch.shape torch.Size([4, 113])
y1_batch.shape <class 'torch.Tensor'> torch.Size([4, 54]) 54
tensor([[14729,    13,  1722,  1783,    55,    37, 12893,    65,     3,     9,
          6780,  2835,     5,   784,   134,  8569,   908, 17482,    10,  3076,
            61, 14729,    13,  1722,  1783,    55,   290,    31,     7,     3,
             9,   443,    16,     8, 12893,    31,     7,  3777,  2071,    44,
             3,     9,  2357,    13,     3,     5,   519,    51,     5,   784,
           134,  8569,   908,     1],
        [14729,    13,  1722,  1783,    55,    37, 12893,    65,     3,     9,
          6780,  2835,     5,   784,   134,  8569,   908, 17482,    10,  3076,
            61, 14729,    13,  1722,  1783,    55,   290,    31,     7,     3,
             9,   443,    16,     8, 12893,    31,     7,  3777,  2071,    44,
             3,     9,  2357,    13,     3,     5,   519,    51,     5,   784,
           134,  8569,   908,     1],
        [14729,    13,  1722,  1783,  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


loss tensor(31.7115, device='cuda:6', grad_fn=<NllLossBackward0>)


In [25]:
batch_size = 32
n_epoch = 200
# total_batch = math.ceil(len(related_sensor_data_li) / batch_size)
# dev_total_batch = math.ceil(len(templates_li) / batch_size)
total_batch = len(train_sensor_data)
dev_total_batch = len(test_sensor_data)
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)

test_losses_epoch=[]
bleu_score_epoch=[]
rouge_score_epoch=[]
meteor_score_epoch=[]

from datasets import load_metric

bleu=load_metric("bleu")
rouge=load_metric("rouge")
meteor = load_metric('meteor')

for epoch in range(n_epoch):
    print('epoch', epoch)

    all_true_labels = []
    all_pred_labels = []
    train_losses = []
    pbar = tqdm(enumerate(generate_data(n_tokens, train_sensor_data, train_text, train_oneshot_data, train_mapping, tokenizer=tokenizer)), total=total_batch)
    for i, (x_batch, y1_batch, y2_batch, m_batch, dii_batch) in pbar:
#         all_true_labels += true_labels
        
        optimizer.zero_grad()
        sequence_ids=model(input_ids=x_batch, attention_mask=m_batch, decoder_input_ids=dii_batch)
                
        output_ids=sequence_ids.logits[:, n_tokens:, :]
        max_pred_ids=sequence_ids.logits[:, n_tokens:, :].argmax(-1).detach()
        
        flat_output_ids=torch.empty((0, output_ids.shape[2]), dtype=output_ids.dtype).to(device)
        flat_max_pred_ids=torch.empty((0,), dtype=max_pred_ids.dtype).to(device)
        for i, (max_pred_ids_unbatch, output_ids_unbatch) in enumerate(zip(max_pred_ids, output_ids)):
            max_pred_ids_tmp=max_pred_ids_unbatch[max_pred_ids_unbatch!=0]
            output_ids_tmp=output_ids_unbatch[max_pred_ids_unbatch!=0]
        
            flat_max_pred_ids=torch.cat((flat_max_pred_ids, max_pred_ids_tmp), 0)
            flat_output_ids=torch.cat((flat_output_ids, output_ids_tmp), 0)

        label_ids=torch.tensor(y1_batch[0], dtype=torch.long).to(device)
        max_length=max(len(flat_output_ids), len(label_ids))
        
        pad_width_pred=max_length-len(flat_output_ids)
        pad_width_label=max_length-len(label_ids)
                
        flat_output_ids=F.pad(flat_output_ids, (0, 0, 0, pad_width_pred), mode='constant', value=0)
        label_ids=F.pad(label_ids, (0, pad_width_label), mode='constant', value=0)
                
        loss=ce_loss(flat_output_ids, label_ids)
        loss.backward()
        optimizer.step()
        
        loss=float(loss.detach().cpu().numpy())
        train_losses.append(loss)

#         acc = accuracy_score(all_true_labels, all_pred_labels)
        pbar.set_description(f'train: loss={np.mean(train_losses):.4f}')

    all_true_labels = []
    all_pred_labels = []
    output_max_pred=[]
    test_losses = []
    
    with torch.no_grad():
        pbar = tqdm(enumerate(generate_data(n_tokens, test_sensor_data, test_text, test_oneshot_data, test_mapping, tokenizer=tokenizer)), total=dev_total_batch)
        for i, (x_batch, y1_batch, y2_batch, m_batch, dii_batch) in pbar:
#             all_true_labels += true_labels
            sequence_ids=model(input_ids=x_batch, attention_mask=m_batch, decoder_input_ids=dii_batch)
            
            output_ids=sequence_ids.logits[:, n_tokens:, :]
            max_pred_ids=sequence_ids.logits[:, n_tokens:, :].argmax(-1).detach()
                    
            # concatenate all sentences_ids and coresponding output_ids
            flat_output_ids=torch.empty((0, output_ids.shape[2]), dtype=output_ids.dtype).to(device)
            flat_max_pred_ids=torch.empty((0,), dtype=max_pred_ids.dtype).to(device)
            for i, (max_pred_ids_unbatch, output_ids_unbatch) in enumerate(zip(max_pred_ids, output_ids)):
                max_pred_ids_tmp=max_pred_ids_unbatch[max_pred_ids_unbatch!=0]
                output_ids_tmp=output_ids_unbatch[max_pred_ids_unbatch!=0]

                flat_max_pred_ids=torch.cat((flat_max_pred_ids, max_pred_ids_tmp), 0)
                flat_output_ids=torch.cat((flat_output_ids, output_ids_tmp), 0)
            
            label_ids=torch.tensor(y1_batch[0], dtype=torch.long).to(device)
                    
            # Pad them to the same length
            max_length=max(len(flat_output_ids), len(label_ids))

            pad_width_pred=max_length-len(flat_output_ids)
            pad_width_label=max_length-len(label_ids)
                    
            flat_output_ids=F.pad(flat_output_ids, (0, 0, 0, pad_width_pred), mode='constant', value=0)
            label_ids=F.pad(label_ids, (0, pad_width_label), mode='constant', value=0)
                    
            label=tokenizer.decode(label_ids)
            output_max_pred=tokenizer.decode(flat_max_pred_ids)
                    
            loss=ce_loss(flat_output_ids, label_ids)
            loss=float(loss.detach().cpu().numpy())
            test_losses.append(loss)
                    
            pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')
            
#     bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
#     bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]
    
#     meteor_output = [meteor_score.single_meteor_score(
#         tokenizer.tokenize(ref_s), tokenizer.tokenize(pred_s), alpha=0.9, beta=3, gamma=0.5)
#                      for ref_s, pred_s in zip(output_max_pred, test_templates)
#                     ]
#     meteor_avg_score=sum(meteor_output) / len(meteor_output)
    
#     bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
#     rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)
    
#     bleu_score_epoch.append(bleu_score)
#     rouge_score_epoch.append(rouge_score)
#     meteor_score_epoch.append(meteor_avg_score)
    
#     test_losses_epoch.append(np.mean(test_losses))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  app.launch_new_instance()
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


epoch 0


train: loss=10.3775:  14%|████▊                              | 25/180 [00:19<02:01,  1.28it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB (GPU 6; 31.75 GiB total capacity; 14.70 GiB already allocated; 58.69 MiB free; 16.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print("test_sensor_data: ", len(test_sensor_data))
# print("all_pred_labels:", all_pred_labels, len(all_pred_labels), len(all_pred_labels[0]))
print("output_max_pred:", output_max_pred[0], len(output_max_pred))
# print(len(test_losses), len(test_sensor_data)/32, len(test_losses_epoch[0]))

# print("bleu_score_epoch", bleu_score_epoch)
# print("rouge_score_epoch", rouge_score_epoch)

In [ ]:
rouge1_precision_epoch=[round(rouge_output["rouge1"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rouge1_recall_epoch=[round(rouge_output["rouge1"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rouge1_fmeasure_epoch=[round(rouge_output["rouge1"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rouge2_precision_epoch=[round(rouge_output["rouge2"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rouge2_recall_epoch=[round(rouge_output["rouge2"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rouge2_fmeasure_epoch=[round(rouge_output["rouge2"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rougeL_precision_epoch=[round(rouge_output["rougeL"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rougeL_recall_epoch=[round(rouge_output["rougeL"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rougeL_fmeasure_epoch=[round(rouge_output["rougeL"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rougeLsum_precision_epoch=[round(rouge_output["rougeLsum"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rougeLsum_recall_epoch=[round(rouge_output["rougeLsum"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rougeLsum_fmeasure_epoch=[round(rouge_output["rougeLsum"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

            
bleu_avg_score_epoch= [round(bleu_output["bleu"], 4) for bleu_output in bleu_score_epoch]
bleu_1_score_epoch=[round(bleu_output["precisions"][0], 4) for bleu_output in bleu_score_epoch]
bleu_2_score_epoch=[round(bleu_output["precisions"][1], 4) for bleu_output in bleu_score_epoch]
bleu_3_score_epoch=[round(bleu_output["precisions"][2], 4) for bleu_output in bleu_score_epoch]
bleu_4_score_epoch=[round(bleu_output["precisions"][3], 4) for bleu_output in bleu_score_epoch]

meteor_score_epoch=[round(meteor_avg, 4) for meteor_avg in meteor_score_epoch]

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'rouge1_precision_epoch': rouge1_precision_epoch,
    'rouge1_recall_epoch': rouge1_recall_epoch,
    'rouge1_fmeasure_epoch': rouge1_fmeasure_epoch,

    'rouge2_precision_epoch': rouge2_precision_epoch,
    'rouge2_recall_epoch': rouge2_recall_epoch,
    'rouge2_fmeasure_epoch': rouge2_fmeasure_epoch,

    'rougeL_precision_epoch': rougeL_precision_epoch,
    'rougeL_recall_epoch': rougeL_recall_epoch,
    'rougeL_fmeasure_epoch': rougeL_fmeasure_epoch,

    'rougeLsum_precision_epoch': rougeLsum_precision_epoch,
    'rougeLsum_recall_epoch': rougeLsum_recall_epoch,
    'rougeLsum_fmeasure_epoch': rougeLsum_fmeasure_epoch,

    'bleu_avg_score_epoch': bleu_avg_score_epoch,
    'bleu_1_score_epoch': bleu_1_score_epoch,
    'bleu_2_score_epoch': bleu_2_score_epoch,
    'bleu_3_score_epoch': bleu_3_score_epoch,
    'bleu_4_score_epoch': bleu_4_score_epoch,
    
    'meteor_score_epoch': meteor_score_epoch,
})

df.to_csv('metrics_epoch_soft_prompt_1shot.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

plt.plot(test_losses_epoch, '*-r')

plt.plot(rouge1_precision_epoch, '*--g')
plt.plot(rouge1_recall_epoch, 'x--g')
plt.plot(rouge1_fmeasure_epoch, 'o--g')

plt.plot(bleu_avg_score_epoch, '*:b')
plt.plot(bleu_1_score_epoch, 'x:b')
plt.plot(bleu_2_score_epoch, '+:b')
plt.plot(bleu_3_score_epoch, 'o:b')
plt.plot(bleu_4_score_epoch, 'd:b')

plt.plot(meteor_score_epoch, '*-.y')

plt.show()

In [ ]:
inputs = tokenizer(test_sensor_data[0], return_tensors="pt").to(device)
print(test_sensor_data[0])

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1).to(device), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1).to(device), inputs['attention_mask']], 1)
inputs['decoder_input_ids']=torch.full((1, 100), 1).to(device)
 
outputs = model(**inputs)
print(outputs.logits.shape)
output_ids=outputs.logits[:, n_tokens:, :]
print(output_ids.shape)
max_ids=output_ids.argmax(-1).detach().cpu().numpy().tolist()
# max_ids[0]=[ids for ids in max_ids[0] if ids!=0]
print(tokenizer.batch_decode(max_ids))

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m_%H%M%S")
torch.save(model.state_dict(), "./output/T5_"+dt_string)

In [ ]:
model = MyModel(num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
output_model = './models/model_xlnet_mid.pth'

# save
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

# load
checkpoint = torch.load(output_model, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model.load_state_dict(torch.load("./output/T5_2404_092246"))
tokenizer = T5Tokenizer.from_pretrained("t5-base")
n_tokens = len(WARNING_DL)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.to(device)

In [ ]:
from datasets import load_metric

In [ ]:
all_true_labels = []
all_pred_labels = []
output_max_pred=[]
test_losses = []
with torch.no_grad():
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, test_sensor_data, test_templates)), total=dev_total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        
        output_ids=outputs.logits[:, n_tokens:, :]
        max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
        max_pred_ids[0]=[ids for ids in max_pred_ids[0] if ids!=0]
        output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
        
        loss = outputs.loss
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        test_losses.append(loss_value)
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels
        pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')

In [ ]:
print(output_max_pred)

In [ ]:
bleu=load_metric("bleu")
rouge=load_metric("rouge")

In [ ]:
bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]


In [ ]:
bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
print(bleu_score)
rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)
print(rouge_score)

In [ ]:
tokenizer.encode("Hello world")